<a href="https://colab.research.google.com/github/ZeroGravigra/MADT8101-Customer-Analytics/blob/main/Topic_Modeling_xplORe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Load libraries and Data


In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis


In [ ]:
!pip install sefr_cut

In [2]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import numpy as np
import re
import string
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx


import pythainlp
from pythainlp.tokenize import sent_tokenize, word_tokenize
import gensim

import sefr_cut
sefr_cut.load_model(engine='tl-deepcut-ws1000')
# sefr_cut.load_model(engine='ws1000')


# import tensorflow_hub as hub
# import tensorflow_text
# import tensorflow as tf

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

loading model.....
Success


In [4]:
df = pd.read_csv('/content/Review App Store - xplORe Touch for more life.csv')

In [5]:
df

,Review_ID,Review
0,1,จ่ายผ่านบัตรเครดิตไม่ได้ ระบบควรพร้อมก่อนที่จะ...
1,2,ใช้แอพผูกกับบัตรบลูการ์ดเอาไว้กดแลกคูปองส่วนลด...
2,3,ระบบเด้ง/ ค้าง หลังอัพเดทล่าสุด
3,4,โคดกาก กดยอมรับเงื่อนไขตอนลงทะเบียนแล้วไม่ไป
4,5,มี Blue Card ใช้แลกรับส่วนลดชานมไข่มุก Kamu ได...
5,6,ใช้บัตร Blue Card แลกคะแนนรับส่วนลด bbq plaza ...
6,7,ใช้แอพเชื่อมกับบัตร Blue card ไว้สะสมคะแนน สาม...
7,8,แต้มบลูการ์ดใช้แลกโปรส่วนลดตรวจสุขภาพได้เยอะมาก
8,9,จะลงทะเบียน e-wallet ในรายละเอียดขั้นตอนการลงท...
9,10,ในแอพแจ้งว่าผูกได้แล้ว แต่จริงๆ ผูกไม่ได้ ทีหล...


Tokenize Words

In [6]:
# sentence & word tokenization
df['Review_tokenized'] = df['Review'].apply(lambda x: [ w for st in sent_tokenize(x, engine='whitespace+newline') for w in sefr_cut.tokenize(st,k=200)])

# flatten & remove stop words
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', '    ', 'ๆ', '/', '>', 'วอลเลท', 'Blue','blue', 'ผูกวอลเลท', 'บัตร', 'e', '-', 'bank', 'รพ', 'ไข่','มุก', 'ม', 'ค่า', 'ตอน', '10','%', '20','บาท', '31','พ.ย.' ,'ปอง']
screening_words = stopwords + removed_words

df['Review_tokenized'] = df['Review_tokenized'].apply(lambda x: [ w for sublist in x for w in sublist if w not in screening_words])

1/1 [==============================] - 0s 47ms/step


In [7]:
# replace after tokenization
replace_dict = {
    'connect' : 'Blue conncet',
    'บลูการ์ด' : 'Blue card',
    'Card' : 'Blue card',
    'card' : 'Blue card',
    'ผูก' : 'เชื่อม' ,
    'แอป' : 'แอพ' ,
    'แล' : 'แลก',
    'อัพเด' : 'อัพเดท',
    'ทล่าสุด' : 'ล่าสุด',
    'ทะเบียน' : 'ลงทะเบียน',
    'แอพเชื่อม' : 'เชื่อม',
    'แนน' : 'คะแนน',
    'แต้' : 'แต้ม',
    '-wallet' : 'e-wallet',
    'k' : 'k-bank',
    'ห้อง' : 'ค่าห้องโรงพยาบาล',
    'ยา' : 'ค่ายา',
    'ชานม' : 'ชานมไข่มุก',
    'ดีลดี' : 'ดีล',
    'คู' : 'ดีล'
}
df['Review_tokenized2'] = df['Review_tokenized'].apply(lambda x: [ replace_dict.get(w) if w in replace_dict else w for w in x ])

In [8]:
df

,Review_ID,Review,Review_tokenized,Review_tokenized2
0,1,จ่ายผ่านบัตรเครดิตไม่ได้ ระบบควรพร้อมก่อนที่จะ...,"[จ่าย, เครดิต, ระบบ, บัง, คับ, ย้าย, แอป, เผลอ...","[จ่าย, เครดิต, ระบบ, บัง, คับ, ย้าย, แอพ, เผลอ..."
1,2,ใช้แอพผูกกับบัตรบลูการ์ดเอาไว้กดแลกคูปองส่วนลด...,"[แอพ, ผูก, บลูการ์ด, กด, แลก, คูปอง, ลด, ห้อง,...","[แอพ, เชื่อม, Blue card, กด, แลก, คูปอง, ลด, ค..."
2,3,ระบบเด้ง/ ค้าง หลังอัพเดทล่าสุด,"[ระบบ, เด้ง, ค้าง, อัพเด, ทล่าสุด]","[ระบบ, เด้ง, ค้าง, อัพเดท, ล่าสุด]"
3,4,โคดกาก กดยอมรับเงื่อนไขตอนลงทะเบียนแล้วไม่ไป,"[โคดกาก, กด, เงื่อนไข, ทะเบียน]","[โคดกาก, กด, เงื่อนไข, ลงทะเบียน]"
4,5,มี Blue Card ใช้แลกรับส่วนลดชานมไข่มุก Kamu ได...,"[Card, แลก, ลด, ชานม, Kamu]","[Blue card, แลก, ลด, ชานมไข่มุก, Kamu]"
5,6,ใช้บัตร Blue Card แลกคะแนนรับส่วนลด bbq plaza ...,"[Card, แลก, คะแนน, ลด, bbq, plaza]","[Blue card, แลก, คะแนน, ลด, bbq, plaza]"
6,7,ใช้แอพเชื่อมกับบัตร Blue card ไว้สะสมคะแนน สาม...,"[แอพเชื่อม, card, สะสม, แนน, แลก, ลด, ตั๋ว, หนัง]","[เชื่อม, Blue card, สะสม, คะแนน, แลก, ลด, ตั๋ว..."
7,8,แต้มบลูการ์ดใช้แลกโปรส่วนลดตรวจสุขภาพได้เยอะมาก,"[แต้, บลูการ์ด, แลก, โปรส่วน, ลด, ตรวจ, สุขภาพ]","[แต้ม, Blue card, แลก, โปรส่วน, ลด, ตรวจ, สุขภาพ]"
8,9,จะลงทะเบียน e-wallet ในรายละเอียดขั้นตอนการลงท...,"[ทะเบียน, -wallet, รายละเอียด, ขั้นตอน, ทะเบีย...","[ลงทะเบียน, e-wallet, รายละเอียด, ขั้นตอน, ลงท..."
9,10,ในแอพแจ้งว่าผูกได้แล้ว แต่จริงๆ ผูกไม่ได้ ทีหล...,"[แอพ, แจ้ง, ผูก, ผูก, ทดสอบ, ชัวร์, เสียหน้า]","[แอพ, แจ้ง, เชื่อม, เชื่อม, ทดสอบ, ชัวร์, เสีย..."


In [9]:
texts = df['Review_tokenized2'].to_list()
#texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [10]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

Topic Modeling

In [11]:
num_topics = 4
chunksize = 2000 # size of the doc looked at every pass
passes = 20 # number of passes through documents #10
iterations = 50
eval_every = 1 # Don't evaluate model perplexity, takes too much time

# Make a index to word dictionary.
temp = dictionary[0] # This is only to "load" the dictionary
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)


CPU times: user 150 ms, sys: 4.06 ms, total: 154 ms
Wall time: 159 ms


In [22]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.116294  0.022994       1        1  41.398489
0      0.093032  0.089747       2        1  23.778737
3      0.065018 -0.097243       3        1  19.826981
2     -0.041756 -0.015498       4        1  14.995792, topic_info=                Term      Freq     Total Category  logprob  loglift
24               แลก  3.000000  3.000000  Default  30.0000  30.0000
23                ลด  4.000000  4.000000  Default  29.0000  29.0000
18         Blue card  4.000000  4.000000  Default  28.0000  28.0000
10            เชื่อม  4.000000  4.000000  Default  27.0000  27.0000
9             เครดิต  2.000000  2.000000  Default  26.0000  26.0000
..               ...       ...       ...      ...      ...      ...
47           ขั้นตอน  0.119853  1.300299   Topic4  -5.2145  -0.4867
45            k-bank  0.119853  1.300299   Topic4  -5.2145  -0.4867
49        รายละเอียด  0.119850  1.300300   Topic4  -5.2145  -0.4867
22  ค่าห้องโรงพยาบาล  0.119850  1.181633   Topic4  -5.2145  -0.3910
50                หา  0.119850  1.300301   Topic4  -5.2145  -0.4867

[185 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
18        1  0.207006     Blue card
18        2  0.621018     Blue card
18        3  0.207006     Blue card
0         1  0.769037  Blue conncet
31        1  0.526980          Kamu
...     ...       ...           ...
63        3  0.885434           โปร
42        2  0.846277       โปรส่วน
81        3  0.597225          โหลด
16        1  0.769037           ใส่
17        1  0.769037       ไว้เล็ก

[108 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3])

In [18]:
model.show_topic(0)

[('แลก', 0.08968575),
 ('Blue card', 0.08938747),
 ('ลด', 0.08938529),
 ('เชื่อม', 0.03844386),
 ('คะแนน', 0.038386673),
 ('ตั๋ว', 0.021359637),
 ('สุขภาพ', 0.021359025),
 ('ตรวจ', 0.021359013),
 ('สะสม', 0.021359012),
 ('โปรส่วน', 0.021359006)]

In [19]:
df['topic'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x))[0][1])

In [23]:
df

,Review_ID,Review,Review_tokenized,Review_tokenized2,topic,score
0,1,จ่ายผ่านบัตรเครดิตไม่ได้ ระบบควรพร้อมก่อนที่จะ...,"[จ่าย, เครดิต, ระบบ, บัง, คับ, ย้าย, แอป, เผลอ...","[จ่าย, เครดิต, ระบบ, บัง, คับ, ย้าย, แอพ, เผลอ...",1,0.989536
1,2,ใช้แอพผูกกับบัตรบลูการ์ดเอาไว้กดแลกคูปองส่วนลด...,"[แอพ, ผูก, บลูการ์ด, กด, แลก, คูปอง, ลด, ห้อง,...","[แอพ, เชื่อม, Blue card, กด, แลก, คูปอง, ลด, ค...",0,0.963772
2,3,ระบบเด้ง/ ค้าง หลังอัพเดทล่าสุด,"[ระบบ, เด้ง, ค้าง, อัพเด, ทล่าสุด]","[ระบบ, เด้ง, ค้าง, อัพเดท, ล่าสุด]",0,0.018802
3,4,โคดกาก กดยอมรับเงื่อนไขตอนลงทะเบียนแล้วไม่ไป,"[โคดกาก, กด, เงื่อนไข, ทะเบียน]","[โคดกาก, กด, เงื่อนไข, ลงทะเบียน]",0,0.018802
4,5,มี Blue Card ใช้แลกรับส่วนลดชานมไข่มุก Kamu ได...,"[Card, แลก, ลด, ชานม, Kamu]","[Blue card, แลก, ลด, ชานมไข่มุก, Kamu]",0,0.941487
5,6,ใช้บัตร Blue Card แลกคะแนนรับส่วนลด bbq plaza ...,"[Card, แลก, คะแนน, ลด, bbq, plaza]","[Blue card, แลก, คะแนน, ลด, bbq, plaza]",0,0.963772
6,7,ใช้แอพเชื่อมกับบัตร Blue card ไว้สะสมคะแนน สาม...,"[แอพเชื่อม, card, สะสม, แนน, แลก, ลด, ตั๋ว, หนัง]","[เชื่อม, Blue card, สะสม, คะแนน, แลก, ลด, ตั๋ว...",0,0.963772
7,8,แต้มบลูการ์ดใช้แลกโปรส่วนลดตรวจสุขภาพได้เยอะมาก,"[แต้, บลูการ์ด, แลก, โปรส่วน, ลด, ตรวจ, สุขภาพ]","[แต้ม, Blue card, แลก, โปรส่วน, ลด, ตรวจ, สุขภาพ]",0,0.963772
8,9,จะลงทะเบียน e-wallet ในรายละเอียดขั้นตอนการลงท...,"[ทะเบียน, -wallet, รายละเอียด, ขั้นตอน, ทะเบีย...","[ลงทะเบียน, e-wallet, รายละเอียด, ขั้นตอน, ลงท...",1,0.984886
9,10,ในแอพแจ้งว่าผูกได้แล้ว แต่จริงๆ ผูกไม่ได้ ทีหล...,"[แอพ, แจ้ง, ผูก, ผูก, ทดสอบ, ชัวร์, เสียหน้า]","[แอพ, แจ้ง, เชื่อม, เชื่อม, ทดสอบ, ชัวร์, เสีย...",0,0.011641
